# **Прогнозирование стоимости автомобиля по характеристикам**



# 1. Импорт библиотек и подготовка

In [ ]:
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor
import xgboost as xgb

from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 1

In [ ]:
# метрика для оценки качества моделей:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# 2. Сбор данных и создание объединенного датасета

In [ ]:
# Адреса директорий:
DIR_TRAIN0  = '../input/parsing-all-moscow-auto-ru-09-09-2020/'
DIR_TRAIN1 = '../input/final-car-price-prediction-df-parsed-sep-2021/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

In [ ]:
!ls '../input'

In [ ]:
# Загрузка данных:
train0 = pd.read_csv(DIR_TRAIN0+'all_auto_ru_09_09_2020.csv') # Октябрь 2021 из Baseline
train1 = pd.read_csv(DIR_TRAIN1+'train_df_full_part1.csv') # Октябрь 2021
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
# Посмотрим что мы имеем в общем виде:
train0.info()
train1.info()
test.info()

In [ ]:
# Объединим все тестовые датасеты в один по принципу 'outer' и посмотрим на итоговый датасет:
combined_check = pd.concat([train0, train1], join='outer', ignore_index=True)
combined_check.info()

In [ ]:
# Приведём столбцы к общим названиям взяв за основу названия из тестового датасета:

for train in [train0, train1, test]:
    train.rename(columns={'Владельцы': 'owners', 'Комплектация': 'complectation_dict',
                          'Владение': 'ownership', 'ПТС': 'vehicle_licence','Привод': 'driving_gear', 'Руль': 'steering_wheel',
                          'Состояние': 'condition', 'Таможня': 'customs'}, inplace=True)
    


In [ ]:
    
for train in [train0, train1, test]:
    train.rename(columns={'model': 'model_name', 'car_url' : 'url'}, inplace=True)


In [ ]:
# Снова объединим трейновые датасеты (теперь с подправленными названиями столбцов):
train = pd.concat([train0, train1], join='outer', ignore_index=True)
train.info()

In [ ]:
# Пометим трейновый и тестовый датасеты и добавим столбец price к тестовому датасету заполнив его нолями:
test['price'] = 0.0
train['train'] = 1
test['train'] = 0


In [ ]:
# Объединим трейновый и тестовый датасеты в один общий по принципу 'inner' (отбросив лишние столбцы):
comb_df = pd.concat([train, test], join='inner', ignore_index=True)
comb_df.info()

# 3. Обработка данных и EDA

## 3.1. Первичный EDA (Pandas profiling)

In [ ]:
from pandas_profiling import ProfileReport


In [ ]:
## Сделаем первичный EDA с помощью Pandas profiling
profile = ProfileReport(comb_df, title = 'Pandas Profiling Report')
profile

In [ ]:
#...выводы о данных

## 3.2. Очистка данных

Переберем все столбцы по порядку и подчистим значения.

*столбец bodyType*

In [ ]:
comb_df.bodyType.value_counts(normalize=True)

In [ ]:
comb_df.bodyType = comb_df.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
comb_df.bodyType.value_counts(normalize=True)

*столбец brand*

In [ ]:
comb_df.brand.value_counts(normalize=True)

In [ ]:
comb_df.brand = comb_df.brand.apply(lambda x: x.lower() if isinstance(x, str) else x)
comb_df.brand.value_counts(normalize=True)

*столбец color*

In [ ]:
comb_df.color.value_counts()

In [ ]:
# заменим цифровые наименования цветов более понятными:
color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий', '200204': 'коричневый',
              'EE1D19': 'красный',  '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый', 
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}

comb_df.color.replace(to_replace=color_dict, inplace=True)
comb_df.color.value_counts(normalize=True)

*столбец fuelType*

In [ ]:
comb_df.fuelType.value_counts(normalize=True)

In [ ]:
comb_df.fuelType = comb_df.fuelType.apply(lambda x: x.lower().strip() if isinstance(x, str) else x)
comb_df.fuelType = comb_df.fuelType.apply(lambda x: (x.replace('универсал', "бензин")) if isinstance(x, str) else x)
comb_df.fuelType = comb_df.fuelType.apply(lambda x: (x.replace('гибрид', "бензин")) if isinstance(x, str) else x)
comb_df.fuelType.value_counts(normalize=True)

*столбец name*

In [ ]:
comb_df.name.value_counts()

In [ ]:
comb_df.model_name.value_counts()

In [ ]:
comb_df.enginePower.value_counts()

Столбец name повторяет столбцы model_name и engine_power. Удалим его предварительно передав данные для заполнения пропусков столбца model_name

In [ ]:
comb_df.model_name.fillna(comb_df.name, inplace=True)
comb_df.drop(['name'], axis=1, inplace=True)

*столбец numberOfDoors*

In [ ]:
comb_df.numberOfDoors.value_counts()

*столбец productionDate*

In [ ]:
comb_df.productionDate.value_counts()

*столбец vehicleConfiguration*

In [ ]:
comb_df.vehicleConfiguration.value_counts()

Столбец vehicleConfiguration повторяет столбцы vehicleTransmission, engineDisplacement, bodyType, numberOfDoors. Удалим его. И сразу посмотрим столбец vehicleTransmission

In [ ]:
comb_df.drop(['vehicleConfiguration'], axis=1, inplace=True)
comb_df.vehicleTransmission.value_counts()

In [ ]:
comb_df.vehicleTransmission.value_counts(normalize=True)

In [ ]:
# создадим и передадим словарь для переименований:
transmission_dict = {'AUTOMATIC': 'автоматическая', 'MECHANICAL' : 'механическая',
                        'ROBOT' : 'роботизированная', 'VARIATOR' : 'вариатор'}

comb_df.vehicleTransmission.replace(to_replace=transmission_dict, inplace=True)
comb_df.vehicleTransmission.value_counts(normalize=True)

In [ ]:
Уже лучше!

*столбец engineDisplacement*

In [ ]:
comb_df.engineDisplacement.value_counts(normalize=True)

In [ ]:
def engineDisplacement_to_float(item):
    try:
        return float(item)
    except:
        return 2.0 # наиболее часто встречающийся объем двигателя
    
def cc_to_ltr(item):
    if item > 10:
        return round((item/1000), 1)
    else:
        return item
    
comb_df.engineDisplacement = comb_df.engineDisplacement.apply(lambda x: x.replace(" LTR", "0.0 LTR") if x == " LTR" else x)
comb_df.engineDisplacement = comb_df.engineDisplacement.apply(lambda x: (x.replace("LTR", "")) if isinstance(x, str) else x)
comb_df.engineDisplacement = comb_df.engineDisplacement.apply(lambda x: (x.replace("d", "")) if isinstance(x, str) else x)
comb_df.engineDisplacement = comb_df.engineDisplacement.apply(engineDisplacement_to_float)
comb_df.engineDisplacement = comb_df.engineDisplacement.apply(cc_to_ltr)
comb_df.engineDisplacement.value_counts(normalize=True)

*столбец enginePower*

In [ ]:
comb_df.enginePower.value_counts(normalize=True)

In [ ]:
def enginePower_to_float(item):
    try:
        return float(item)
    except:
        return float(150) # наиболее часто встречающаяся мощность

comb_df.enginePower = comb_df.enginePower.apply(lambda x: x.split()[0].strip() if isinstance(x, str) else x)
comb_df.enginePower = comb_df.enginePower.apply(enginePower_to_float)
comb_df.enginePower.value_counts(normalize=True)

*столбец description*

In [ ]:
comb_df.description.head(5)

In [ ]:
Это описание машин от владельцев- не нужный нам столбец, удалим его 

In [ ]:
comb_df.drop(['description'], axis=1, inplace=True)

*столбец complectation_dict*

In [ ]:
comb_df.complectation_dict.tail(20)

Столбец complectation_dict повторяет информацию других столбцов. Удалим.

In [ ]:
comb_df.drop(['complectation_dict'], axis=1, inplace=True)

*столбец driving_gear*

In [ ]:
comb_df.driving_gear.value_counts()

*столбец steering_wheel*

In [ ]:
comb_df.steering_wheel.value_counts()

In [ ]:
# создадим словарь для переименований:
steering_wheel_dict = {'LEFT': 'Левый', 'RIGHT' : 'Правый'}

# переименуем и уберем заглавные буквы
comb_df.steering_wheel.replace(to_replace=steering_wheel_dict, inplace=True)
comb_df.steering_wheel = comb_df.steering_wheel.apply(lambda x: x.lower() if isinstance(x, str) else x)
comb_df.steering_wheel.value_counts(normalize=True)

*столбец condition*

In [ ]:
comb_df.condition.value_counts()

В дальнейшем "не требует ремонта" придется поменять на 1 и 0

столбец owners

In [ ]:
comb_df.owners.value_counts()

In [ ]:
comb_df.owners.unique()

In [ ]:
# создадим и передадим словарь для переименований:
owners_dict = {'1 владелец': '1.0', '1\xa0владелец': '1.0','2\xa0владельца': '2.0',
               '2 владельца': '2.0', '3 или более' : '3.0'}

comb_df.owners.replace(to_replace=owners_dict, inplace=True)
comb_df.owners = comb_df.owners.apply(lambda x: float(x) if isinstance(x, str) else x)
comb_df.owners.value_counts(normalize=True)

*столбец vehicle_licence*

In [ ]:
comb_df.vehicle_licence.value_counts()

In [ ]:
# создадим и передадим словарь для переименований:
vehicle_licence_dict = {'ORIGINAL': 'Оригинал', 'DUPLICATE' : 'Дубликат'}

comb_df.vehicle_licence.replace(to_replace=vehicle_licence_dict, inplace=True)
comb_df.vehicle_licence = comb_df.vehicle_licence.apply(lambda x: x.lower() if isinstance(x, str) else x)
comb_df.vehicle_licence.value_counts(normalize=True)

*столбец customs*

In [ ]:
comb_df.customs.value_counts()

In [ ]:
comb_df.customs.unique()

In [ ]:
# Столбец бинарен, заменим значения на 0 - нет значений и 1 - растаможен:
comb_df.customs = comb_df.customs.apply(lambda x: 1 if x == 'Растаможен' or x == True else 0)
comb_df.customs.value_counts()

*столбец ownership*

In [ ]:
comb_df.ownership.value_counts()

Cтолбец ownership выдает информацию о времени нахождения у последнего владельца?.Нам она не нужна. Удалим.

In [ ]:
comb_df.drop(['ownership'], axis=1, inplace=True)

*столбец model_name*

In [ ]:
comb_df.model_name.value_counts()

In [ ]:
comb_df.model_name = comb_df.model_name.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
comb_df.model_name.value_counts()

*столбец url*

In [ ]:
comb_df.url.tail(1000)

столбец url бесполезен для нас. Удалим.

In [ ]:
comb_df.drop(['url'], axis=1, inplace=True)

*Столбец image*

In [ ]:
comb_df.image.tail(1000)

столбец image не несёт важной для нас информации. Удалим.

In [ ]:
comb_df.drop(['image'], axis=1, inplace=True)

*столбец model_info*

In [ ]:
comb_df.model_info.tail(100)

In [ ]:
# полезной для нас информации здесь нет, удаляем
comb_df.drop(['model_info'], axis=1, inplace=True)

*столбец parsing_unixtime*

In [ ]:
comb_df.parsing_unixtime.tail(100)

In [ ]:
# Эта информация для нас бесполезна, удаляем
comb_df.drop(['parsing_unixtime'], axis=1, inplace=True)

*столбец priceCurrency*

In [ ]:
comb_df.priceCurrency.value_counts()

In [ ]:
# Нам не нужен, удалим
comb_df.drop(['priceCurrency'], axis=1, inplace=True)

*столбец sell_id*

In [ ]:
comb_df.sell_id.tail(100)

*столбец super_gen*

In [ ]:
comb_df.super_gen.tail(100)

In [ ]:
# повторяется информация из других столбцов, удалим его
comb_df.drop(['super_gen'], axis=1, inplace=True)

*столбец vendor*

In [ ]:
comb_df.vendor.value_counts()

In [ ]:
comb_df.vendor.tail(100)

In [ ]:
# бесполезный столбец, удаляем
comb_df.drop(['vendor'], axis=1, inplace=True)

In [ ]:
# посмотрим что получилось
comb_df.info()

## 3.3. Удаление дубликатов и заполнение пропусков

In [ ]:
# Посмотрим на количество дубликатов и удалим их:
print(sum(comb_df.duplicated()))
print(comb_df.shape)
comb_df.drop_duplicates(inplace=True)
comb_df.shape

In [ ]:
# Посмотрим на пропуски в train и test порциях датасета по отдельности.
print(comb_df[comb_df.train == 0].isna().sum(axis=0) * 100 / comb_df[comb_df.train == 0].shape[0])
print()
print(comb_df[comb_df.train == 1].isna().sum(axis=0) * 100 / comb_df[comb_df.train == 1].shape[0])

Столбец equipment_dict содержит много пропусков (как train так и test порции) и из него тяжело вытянуть полезную информацию. Удалим его. Так же заметим что test порция содержит лишь небольшое количество пропусков vehicle_licence и остальные данные полные.

In [ ]:
comb_df.drop(['equipment_dict'], axis=1, inplace=True)

In [ ]:
# Посмотрим теперь где поконкретнее пропуски в train порции.
sns.heatmap(comb_df[comb_df.train == 1].isna(), cbar=False);

В датасете из baseline отсутствуют столбцы condition и sell_id. Ничего страшного, эти столбцы не являются основополагающими.

Теперь пройдемся по очереди по всем столбцам с пропусками в объединеном датасете comb_df и заполним пропуски

In [ ]:
comb_df.info()

Сперва заметим что важный столбец price (наш таргет) имеет пропуски которые мы не в праве заполнить. Поэтому остается только удалить эти строки. Важно что test порция не имеет пропусков (мы заполнили соответствующие ячеки нулевыми значениями)

In [ ]:
comb_df.dropna(subset=['price'], inplace=True)

In [ ]:
comb_df.info()

In [ ]:
#небольшое количество пропусков в столбце bodyType заполним наиболее популярным значением:
comb_df['bodyType'].fillna('внедорожник', inplace=True)

In [ ]:
# Посмотрим на небольшое количество пропусков в столбце fuelType
comb_df[comb_df.fuelType.isna()]

In [ ]:
# заполним пропуск 
comb_df['fuelType'].fillna('бензин', inplace=True)

Небольшие пропуски в modelDate заменим значениями из столбца productionDate

In [ ]:
comb_df['modelDate'].fillna(comb_df['productionDate'], inplace=True)

Количество дверей не является выжным параметром, заполним пропуски наиболее распространенным вариантом: 5 дверей

In [ ]:
comb_df['numberOfDoors'].fillna(5, inplace=True)

In [ ]:
# Посмотрим на небольшое количество пропусков в столбце vehicleTransmission
comb_df[comb_df.vehicleTransmission.isna()]

Это старые автомобили поэтому заполним пропуски как 'механическая'

In [ ]:
comb_df.vehicleTransmission.fillna('механическая', inplace = True)

In [ ]:
# Посмотрим на небольшое количество пропусков в столбце engineDisplacement
comb_df[comb_df.engineDisplacement.isna()]

Это малолитражка Honda Civic, заполним объем двигателя как 1.6

In [ ]:
comb_df.engineDisplacement.fillna(1.6, inplace=True)

In [ ]:
# Посмотрим на небольшое количество пропусков в столбце enginePower
comb_df[comb_df.enginePower.isna()]

Опять эта машина! Заполним как 100л.с

In [ ]:
comb_df.enginePower.fillna(100, inplace=True)

In [ ]:
# Посмотрим что за пропуски в столбце milage (пробег) и какого года выпуска эти автомобили
display(comb_df[comb_df.mileage.isna()])
comb_df[comb_df.mileage.isna()].productionDate.value_counts()

По всей видимости это новые aвтомобили. Пустые значения столбца mileage (пробег) заполним 0

In [ ]:
comb_df.mileage.fillna(0, inplace=True)

In [ ]:
# Посмотрим что за пропуски в столбце driving_gear и какого типа эти автомобили
display(comb_df[comb_df.driving_gear.isna()])
comb_df[comb_df.driving_gear.isna()].bodyType.value_counts()

Видим что полно внедорожников поэтому заполним пропуски как 'полный' привод

In [ ]:
comb_df.driving_gear.fillna('полный', inplace=True)

Пропуски в steering_wheel заполним как 'левый'

In [ ]:
comb_df.steering_wheel.fillna('левый', inplace=True)

Столбец condition бинарен, заполним значениями 1 если указано что ремонта не требуетсяб иначе 0

In [ ]:
comb_df.condition = comb_df.condition.apply(lambda x: 1 if x == 'Не требует ремонта' else 0)

In [ ]:
# Посмотрим что за пропуски в столбце owner и какого года выпуска эти автомобили
display(comb_df[comb_df.owners.isna()])
comb_df[comb_df.owners.isna()].productionDate.value_counts()

В основном новые авто , заменим пропуски 0

In [ ]:
comb_df.owners.fillna(0, inplace=True)

In [ ]:
# Посмотрим что за пропуски в столбце vehicle_licence и сколько хозяев было у автомобилей
display(comb_df[comb_df.vehicle_licence.isna()])
comb_df[comb_df.vehicle_licence.isna()].owners.value_counts()

Это же в подавляющем большинстве те новые автомобили для которых мы заполниои 0 хозяев. Поэтому заполним пропуски как 'оригинал'

In [ ]:
comb_df.vehicle_licence.fillna('оригинал', inplace=True)

In [ ]:
comb_df.info()

Столбец sell_id в модель подавать не будем но и удалять его не будем потому что он нужен для сабмита. 

## 3.4. EDA

In [ ]:
# Посмотрим количество уникальных значений в каждом столбце
comb_df.nunique(dropna=False)

In [ ]:
# Сгруппируем признаки по типам
num_cols = ['engineDisplacement', 'enginePower', 'mileage', 'modelDate', 'productionDate']
bin_cols = ['condition', 'customs', 'steering_wheel', 'vehicle_licence']
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'model_name', 'driving_gear', 'owners', 'numberOfDoors']
target_cols = ['price']

### 3.4.1. Анализ числовых признаков

In [ ]:
# посмотрим на корреляцию числовых признаков
sns.pairplot(comb_df[num_cols])

Видим сильную корреляцию между modelDate и productionDate что ожидаемо. Есть несколько других закомерностей, например, engine power и productionDate. Есть выбросы.

In [ ]:
# Посмотрим на общую информацию
comb_df[num_cols].describe()

In [ ]:
# Посмотрим на корреляцию числовых признаков с ценой (нашим таргетом)
plt.figure(figsize=(15, 8));
sns.heatmap(comb_df[comb_df.train == 1][num_cols + ['price']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')

Видим что все признаки сильно влияют на цену за исключением engineDisplacement который в свою очередь тесно связан с enginePower. Также modelDate и productionDate показывают сильную взаимосвязь. Удалим engineDisplacement и modelDate

In [ ]:
comb_df.drop(['modelDate', 'engineDisplacement'], axis=1, inplace=True)
for col in ['modelDate', 'engineDisplacement']:
    num_cols.remove(col)

### 3.4.2. Анализ бинарных и категоральных признаков

Посмотрим есть ли дисбаланс между трейновой и тестовой выборками

In [ ]:
for col in (bin_cols + cat_cols):
    if col not in ['model_name']:
        fig, ax = plt.subplots(figsize=(15, 4), ncols=2, nrows=1)
        ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
        ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])

Видно что есть существенные перекосы. Тестовая выборка ограничена. Трейновая выборка гораздо шире. Если конкретнее, то столбцы condition и custom не являются бинарными в тестовой выборке (все строки имеют значения "не требует ремонта" и "растаможен"). Также тестовая выборка содержит лишь 12 марок (brands) и меньше категоральных значений в столбцах fuelType и owners. 
**Удалим лишнее и приведём трейновую часть в соответствие с тестовой. Тут надо чётко понимать что мы это делаем для того чтобы не перегружать моделирование и для улучшения точности предсказания для нашей конкретной тестовой части. В обшем случае мы бы этого не делали.**

In [ ]:
comb_df = comb_df[comb_df.brand.isin(comb_df[comb_df.train == 0].brand.unique())]

In [ ]:
comb_df = comb_df[comb_df.fuelType.isin(comb_df[comb_df.train == 0].fuelType.unique())]

In [ ]:
comb_df = comb_df[comb_df.owners.isin(comb_df[comb_df.train == 0].owners.unique())]

In [ ]:
comb_df.drop(['condition', 'customs'], axis=1, inplace=True)
for col in ['condition', 'customs']:
    bin_cols.remove(col)
comb_df.info()

In [ ]:
Треть записей ушло, за то подстроились под тестовый датасет

In [ ]:
for col in (bin_cols + cat_cols):
    if col not in ['model_name']:
        fig, ax = plt.subplots(figsize=(15, 4), ncols=2, nrows=1)
        ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
        ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])

Ну что же, что то получилось

### 3.4.3. Анализ таргета (price)

In [ ]:
# посмотрим на таргет (price)
comb_df.query('train == 1').price.hist();
plt.title('The target variable distribution', fontdict={'fontsize': 14});
plt.xlabel('price, RUB * 10^7')

Большой хвост по цене, поэтому прологорифмируем наш таргет и добавим его как новый столбец

In [ ]:
np.log2(comb_df.query('train == 1').price).hist();
plt.title('The log2 target variable distribution', fontdict={'fontsize': 14});
comb_df['price_log2'] = np.log2(comb_df.price + 1)

### 3.4.4. Выбросы

In [ ]:
# Посмотрим на выбросы в числовых признаках:

def outliers_iqr(col): #Избавление числового признака от выбросов
        quartile_1, quartile_3 = np.percentile(list(col), [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr * 1.5)
        upper_bound = quartile_3 + (iqr * 1.5)
        return comb_df[col < upper_bound][col > lower_bound]
    
def plot_boxplots(features): # Построение boxpots
    for feature in features:
        plt.figure(figsize=(6,3))
        sns.boxplot(features[feature], color = 'blue')
        print('Среднее значение {} = {:.3f}'.format(feature, features[feature].mean()))
    
    
plot_boxplots(comb_df[num_cols])
    



Вполне нормальные данные по авто, выбросы относятся к раритетным и машинам с мощным двигателем

### 3.4.5. Feature Engineering

In [ ]:
print(num_cols)

Признак productionDate вызывает вопрос. Лучше использовать возраст автомобиля вместо года производства. Также создадим новый признак: средний годовой пробег.

In [ ]:
comb_df['age'] = 2021 - comb_df.productionDate
comb_df['mileage_per_year'] = round(comb_df['mileage'] / comb_df['age'], 0)
num_cols = num_cols+['age','mileage_per_year']
num_cols.remove('productionDate')
comb_df.drop(['productionDate'], axis=1, inplace=True)

Также создадим следующие новые признаки:

1) столбец обозначающий старые автомобили (старше 10 лет) так как цена на них не сильно зависит от возраста

2) столбец обозначающий редкие цвета так как это чвсто дорогие автомобили

3) автомобили с редкими типами кузова которые могут обозначать высокую цену

Признаки эти бинарны

In [ ]:
comb_df['old_car'] = comb_df.age.apply(lambda x: 1 if x >10 else 0)
comb_df['rare_colors'] = comb_df.color.apply(lambda x: 1 if x in ['фиолетовый', 'пурпурный', 'золотистый', 'оранжевый', 'жёлтый', 'розовый'] else 0)
comb_df['rare_bodyType'] = comb_df.bodyType.apply(lambda x: 1 if x in ['микровэн', 'седан-хардтоп', 'лимузин', 'тарга', 'фастбек'] else 0)
bin_cols = bin_cols+['old_car','rare_colors','rare_bodyType']

In [ ]:
# Прологорифмируем числовые признаки
comb_df['enginePower_log2'] = np.log2(comb_df.enginePower+1)
comb_df['enginePower_log2'].replace([np.inf, -np.inf], 0, inplace=True)
comb_df['mileage_log2'] = np.log2(comb_df.mileage+1)
comb_df['mileage_log2'].replace([np.inf, -np.inf], 0, inplace=True)
comb_df['age_log2'] = np.log2(comb_df.age+1)
comb_df['age_log2'].replace([np.inf, -np.inf], 0, inplace=True)
comb_df['mileage_per_year_log2'] = np.log2(comb_df.mileage_per_year+1)
comb_df['mileage_per_year_log2'].replace([np.inf, -np.inf], 0, inplace=True)
# Удалим старые слолбцы
comb_df.drop(['enginePower','mileage','age','mileage_per_year'], axis=1, inplace=True)
num_cols = ['enginePower_log2', 'mileage_log2', 'age_log2', 'mileage_per_year_log2']

Проанализируем

In [ ]:
# Посмотрим на корреляцию новых числовых параметров с ценой (нашим таргетом)
plt.figure(figsize=(15, 8));
sns.heatmap(comb_df[comb_df.train == 1][num_cols + ['price_log2']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')

Могло быть и получше, ну что же

In [ ]:
# Посмотрим на сбалансирование бинарных и категоральных признаков трейнового и тестового датасетов
for col in (bin_cols + cat_cols):
    if col not in ['model_name']:
        fig, ax = plt.subplots(figsize=(15, 4), ncols=2, nrows=1)
        ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
        ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 14})
        comb_df[comb_df.train == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])

In [ ]:
# Сделаем Label Encoding для бинарных и категоральных признаков
for colum in ['steering_wheel', 'vehicleTransmission', 'vehicle_licence', 'driving_gear',
              'fuelType', 'color', 'brand', 'bodyType']:
    comb_df[colum] = comb_df[colum].astype('category').cat.codes

cols_to_encode = list(set(comb_df.columns) & set(cat_cols))
for colum in cols_to_encode:
    comb_df[colum] = comb_df[colum].astype('category').cat.codes

In [ ]:
# Теперь посмотрим на важность бинарных и категоральных признаков
imp_cat = pd.Series(
    mutual_info_regression(
        comb_df[comb_df.train == 1][list(set(comb_df.columns) & set(cat_cols+bin_cols))], 
        comb_df[comb_df.train == 1]['price_log2'], 
        discrete_features=True), index=list(set(comb_df.columns) & set(cat_cols+bin_cols))
)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', title='Важность категоральных и бинарных признаков')
plt.show()

Неплохо . Можно подавать в модели

# 4. Machine Learning

## 4.1. Разделение данных

Разделим трейновые данные на непосредственно трейновые и тестовые. Также обозначим данные для сабмишена

In [ ]:
X = comb_df.query('train == 1').drop(['sell_id', 'price', 'price_log2', 'train'], axis=1)
X_sub = comb_df.query('train == 0').drop(['sell_id', 'price','price_log2', 'train'], axis=1)
y = comb_df.query('train == 1').price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## 4.2. Построение и тестирование моделей

### 4.2.1. Наивная модель

Создадим наивную модель которая будет предсказывать стоимость автомобиля по его возрасту

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам age_log2 в трейне и размечаем значения на тесте
predict = X_test['age_log2'].map(tmp_train.groupby('age_log2')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 49.65%

### 4.2.2. Catboost

Параметры для этой модели были взяты из Baseline.

In [ ]:
catboost = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
catboost.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

# оцениваем точность
predict_catboost = catboost.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_catboost))*100:0.2f}%")

Точность модели по метрике MAPE: 17.66%

Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    

In [ ]:
catboost_log = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
catboost_log.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

predict_catboost_log = np.exp(catboost_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_catboost_log))*100:0.2f}%")

Точность модели по метрике MAPE: 15.52%

Точность после передачи логарифмированного target заметно улучшилась. Будем это использовать в других моделях.


### 4.2.3. LGBMRegressor с подбором гиперпараметров

In [ ]:
# Запустим модель со стандартными параметрами
lgbm_log = LGBMRegressor(random_state=RANDOM_SEED)
lgbm_log.fit(X_train, np.log(y_train))
predict_lgbm_log = np.exp(lgbm_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_lgbm_log))*100:0.2f}%")

Точность модели по метрике MAPE: 17.35%

In [ ]:
## Создадим сетку для подбора гиперпараметров и пройдемся по ней

lgbm_log_param_dic = {
 'learning_rate':[0.005, 0.01, 0.015],
 'num_leaves':[250, 500, 1000],
 'colsample_bytree': [0.5, 1],
 'min_child_samples': [5, 10, 20]
}

lgbm_log_grid = GridSearchCV(lgbm_log, lgbm_log_param_dic, n_jobs=-1,
                             refit=True, cv=2, return_train_score=True, verbose=5)


lgbm_log_grid.fit(X_train, np.log(y_train))
print(lgbm_log_grid.best_params_)

Перебор гиперпараметров занял не один час. В итоге получился следующий результат:

Fitting 2 folds for each of 54 candidates, totalling 108 fits
{'colsample_bytree': 1, 'learning_rate': 0.015, 'min_child_samples': 5, 'num_leaves': 1000}

In [ ]:
# Запустим модель с подобранными гиперпараметрами
lgbm_log = LGBMRegressor(random_state=RANDOM_SEED, objective = 'regression', num_iterations = 1000, 
                           n_estimators = 1000, learning_rate = 0.015, num_leaves = 1000, max_depth = -1,  
                           min_child_samples = 5,  colsample_bytree = 1)
lgbm_log.fit(X_train, np.log(y_train))
predict_lgbm_log = np.exp(lgbm_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_lgbm_log))*100:0.2f}%")

Точность модели по метрике MAPE: 15.93%
Выжать точность побольше не получается скорее всего из за малого количества данных

### 4.2.4. XGBoostRegressor

In [ ]:
# Запустим модель с параметрами подобранными вручную по смыслу (перебор по сетке занимает много времени)
xgb_log = xgb.XGBRegressor(
    objective='reg:squarederror', 
    colsample_bytree= 0.5,               
    learning_rate=0.1, 
    max_depth= 12, 
    alpha=1,                   
    n_estimators=1000,
    random_state=RANDOM_SEED,
    verbose=1, 
    n_jobs=-1)

xgb_log.fit(X_train, np.log(y_train))
predict_xgb_log = np.exp(xgb_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_xgb_log))*100:0.2f}%")

Точность модели по метрике MAPE: 15.72%

Тоже не плохо

### 4.2.5. Random forest 

In [ ]:
# Запустим модель со стандартными параметрами
rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

Точность модели по метрике MAPE: 16.35%

MAE колеблется от 15.72 и выше, подбор гиперпараметров улучшил точность ,но до ТОР далековато (: 

### 4.2.6. Extra Tree Regressor с подбором гиперпараметров

In [ ]:
# Запустим модель со стандартными параметрами
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))
predict_etr_log = np.exp(etr_log.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_etr_log))*100:0.2f}%")

Точность модели по метрике MAPE: 16.97%
Что то получилось, не лучший результат 

In [ ]:
## Подберем гиперпараметры с помощью hyperopt
hp.uniform('n_estimators',100,500),
hp.choice("n_estimators", [int(x) for x in np.linspace(200, 1000, num = 17)])
#
def objective(params):
    model=ExtraTreesRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        min_samples_leaf=int(params['min_samples_leaf']),
        min_samples_split=int(params['min_samples_split']),
        bootstrap=params['bootstrap'],
        max_features=params['max_features'],
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    model.fit(X_train, np.log(y_train))
    pred=model.predict(X_test)
    score=mape(y_test,np.exp(pred))
    return score

def optimize(trial):
    params={
        'n_estimators': hp.uniform('n_estimators',100,500),
        'max_features': hp.choice("max_features", ['auto', 'sqrt']),
        'max_depth': hp.uniform('max_depth',5,15),
        'min_samples_split': hp.uniform('min_samples_split',2,10),
        'min_samples_leaf': hp.uniform('min_samples_leaf',1,5),
        'bootstrap': hp.choice("bootstrap", [True, False])
    }
    best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=100, rstate=np.random.RandomState(RANDOM_SEED))
    return best

trial=Trials()
best=optimize(trial)

In [ ]:
#best #(результаты)

Подбор занял 15 минут и в итоге дал следующий результат:

best = {'bootstrap': 1,
 'max_depth': 14.149743820071635,
 'max_features': 0,
 'min_samples_leaf': 1.9153096840827872,
 'min_samples_split': 4.041822394468241,
 'n_estimators': 194.71822939909046}

In [ ]:
# Запустим модель с подобранными гирерпараметрами
etr_log_hp = ExtraTreesRegressor(random_state=RANDOM_SEED, 
                                   n_jobs=-1, 
                                   verbose=1, 
                                   n_estimators = 195, 
                                   min_samples_split = 4, 
                                   min_samples_leaf = 2, 
                                   max_features = 'auto', 
                                   max_depth = 14, 
                                   bootstrap = 1)

etr_log_hp.fit(X_train, np.log(y_train))
predict_etr_log_hp = np.exp(etr_log_hp.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_etr_log_hp))*100:0.2f}%")

Точность модели по метрике MAPE: 16.85%

Чуть чуть получше

Подбор гиперпараметров не улучшил результат ,скорее всего мало данных и результат не так заметен

### 4.2.7. Stacking

Объединим три модели в одну

In [ ]:
estimators = [
    ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
    ('rf', RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
    ('lgmb', LGBMRegressor(random_state=RANDOM_SEED, objective = 'regression', num_iterations = 1000, n_estimators = 1000, learning_rate = 0.015, num_leaves = 1000, max_depth = -1,  
                           min_child_samples = 5,  colsample_bytree = 1))]

stacking_log = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

stacking_log.fit(X_train, np.log(y_train))

predict_stacking_log = np.exp(stacking_log.predict(X_test))

print(f"The MAPE mertic for the default StackingRegressor model: {(mape(y_test, predict_stacking_log) * 100):0.2f}%.")

Точность модели по метрике MAPE: 15.79%

Stacking выдал средние результаты МАРЕ: результат 15.79% значительно лучше каждой модели по отдельности (etr = 16.97%, rf = 16.35%, lgmb = 15.93%). 

Stacking помог улучшить результат.

Скорее всего на больших количествах данных результат может быть и лучше



# 5. Submission

In [ ]:
VERSION=1
predict_submission = np.exp(etr_log.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)


# 6. Заключение

Вроде получился неплохой результат.Могло быть и лучше если бы было больше данных, жаль не получилось спарсить данные с auto.ru

Точность можно было бы повысить если поработать со столбцом где дана комплектация автомобиля (что то выделить из этой кучи)

Было интересно проверить теорию и увидеть результаты работы на практике
